In [2]:
import pandas as pd
import numpy as np
import librosa
import pathlib as path
import librosa.display
import matplotlib.pyplot as plt
import scipy
from collections import Counter
import collections

In [3]:
song = r"C:/Users/Mahip/Documents/songdata/EDM/girl_like_you.mp3"

In [4]:
y1, sr = librosa.load(song,offset=38, duration=30)

In [5]:
notes_dict = {'0': 'C', '1': 'C#', '2': 'D','3': 'D#','4': 'E','5': 'F','6': 'F#','7': 'G','8': 'G#','9': 'A','10': 'A#','11': 'B'}

In [6]:
def extract_notes(y):

    y_harm = librosa.effects.harmonic(y=y)
    chroma_os_harm = librosa.feature.chroma_cqt(y=y_harm, sr=sr, bins_per_octave=12*3)

    chroma_filter = np.minimum(chroma_os_harm,librosa.decompose.nn_filter(chroma_os_harm,aggregate=np.median,metric='cosine'))
    chroma_smooth = scipy.ndimage.median_filter(chroma_filter, size=(1, 9))
    
    note_list=[]
    for i in range(len(chroma_smooth[0])):
        max_val = max(chroma_smooth[0][i], chroma_smooth[1][i], chroma_smooth[2][i], chroma_smooth[3][i], chroma_smooth[4][i], chroma_smooth[5][i], chroma_smooth[6][i], chroma_smooth[7][i], chroma_smooth[8][i], chroma_smooth[9][i], chroma_smooth[10][i], chroma_smooth[11][i])
        for j in range(12):
            if max_val == chroma_smooth[j][i]:
                note_list+=(notes_dict[str(j)])
                
    i=0
    updated_list =[]
    while(i<len(note_list)-1):
        if note_list[i+1]=='#':
            updated_list.append(str(note_list[i])+str('#'))
            i+=1
        else:
            updated_list.append(note_list[i])
        i+=1
    
    data = Counter(updated_list)
    new = data.most_common(1)
    note = new[0][0]
    
    print('The Most comman notes is: ' + note)

    data_sequence = []
    length = len(updated_list)
    i=0
    while(i!=length-2):
        count=1
        while(updated_list[i]==updated_list[i+1] and i!=length-3):
            count+=1
            i+=1
        if i==length-3:
            data_sequence.append([updated_list[i],count+2])
        else:
            data_sequence.append([updated_list[i],count])
        i+=1
        
    return data_sequence

In [7]:
extracted_notes_list = extract_notes(y1)

The Most comman notes is: A


In [8]:
from collections import defaultdict
#find all notes
def getCommmonNotes(loc, s):
    substr = s[loc:]
    i = -1
    while(substr):
        yield substr
        substr = s[loc:i]
        i -= 1
        
#find repetitive notes
def longestNotesSequence(r):
    occ = defaultdict(int)
    for i in range(len(r)):
        for sub in getCommmonNotes(i,r):
            occ[sub] += 1
    filtered = [k for k,v in occ.items() if v >= 2]
    if filtered:
        maxkey =  max(filtered, key=len) # Find longest string
        return maxkey
    else:
        return "No Notes"

In [9]:
first_c= []

for i in range(len(extracted_notes_list)):
    first_c.append(extracted_notes_list[i][0])
    
first_c_tuple = tuple(first_c)

In [10]:
period=100
everyThousand= int(len(first_c_tuple)/period)
commonNotes=[]

for i in range(everyThousand):
    start=i*period
    end =(i+1)*period -1
    overlap=longestNotesSequence(first_c_tuple[start:end])
    if(len(overlap)<period):
        commonNotes.append(overlap)
            
ctr = collections.Counter(commonNotes)

In [11]:
pd.set_option('display.max_rows',50)
df = pd.DataFrame(list(ctr.items()),columns=['Notes','Occurences'])
df.sort_values('Occurences',ascending=False)

,Notes,Occurences


In [12]:
extracted_notes_list

[['F', 51],
 ['C', 95],
 ['G', 74],
 ['A', 99],
 ['F', 74],
 ['C', 80],
 ['B', 2],
 ['G', 71],
 ['A', 102],
 ['F', 75],
 ['C', 82],
 ['G', 76],
 ['B', 1],
 ['G', 3],
 ['A', 42],
 ['E', 8],
 ['A', 31],
 ['E', 1],
 ['A', 1],
 ['F', 85],
 ['C', 82],
 ['G', 81],
 ['A', 40],
 ['E', 10],
 ['A', 25]]

In [13]:
extract_notes(y1)

The Most comman notes is: A


[['F', 51],
 ['C', 95],
 ['G', 74],
 ['A', 99],
 ['F', 74],
 ['C', 80],
 ['B', 2],
 ['G', 71],
 ['A', 102],
 ['F', 75],
 ['C', 82],
 ['G', 76],
 ['B', 1],
 ['G', 3],
 ['A', 42],
 ['E', 8],
 ['A', 31],
 ['E', 1],
 ['A', 1],
 ['F', 85],
 ['C', 82],
 ['G', 81],
 ['A', 40],
 ['E', 10],
 ['A', 25]]